# WGCNA

+ Author: Margaret Paiva
+ Date: 2021-12-11
+ Output: R notebook

WGCNA defines co-expression networks as undirected, weighted gene networks. The nodes of such a network correspond to gene expression profiles, and edges between genes are determined by the pairwise correlations between gene expression. It alleviates the multiple testing problem and focuses on the relationship between a few modules and the sample trait. Toward this end, it calculates the eigengene significance (correlation between sample trait and eigengene) and the corresponding p-value for each module. The module definition does not make use of a priori defined gene sets. Instead, modules are constructed from the expression data by using hierarchical clustering.

## 1. Dependencies

In [1]:
# cran packages
x <- c('dplyr',
       'data.table',
       'tidyr',   
       'tidyverse', 
       'tibble',
       'ggplot2', 
       'cluster', 
       'WGCNA')
# bioconductor packages
y <- c('GO.db')

In [2]:
# install cran packages
for (pkg in x) {
    if (!pkg %in% rownames(installed.packages())) {install.packages(pkg)}
}

In [8]:
# install bioconductor packages
if (!requireNamespace("BiocManager", quietly=TRUE))
    install.packages("BiocManager")
for (pkg in y) {
    if (!requireNamespace(pkg, quietly=TRUE)) {
        BiocManager::install(pkg)
    }
}

In [4]:
# load packages
load_lib <- function(x) {
    suppressPackageStartupMessages(library(x, character.only = TRUE))
}
invisible(lapply(c(x, y), load_lib))

In [5]:
allowWGCNAThreads()   
options(stringsAsFactors = FALSE)

Allowing multi-threading with up to 8 threads.


### 2. Request data from API
Request data from API using the files at https://github.com/Champions-Oncology/Workspaces/tree/main/1starter_data_request.

This notebook used the following options to request data:

(This notebook used [this example list of genes](https://github.com/Champions-Oncology/Workspaces/blob/main/gene_list.csv). You may define your genes of interest in a .csv file and use it in the data requesting file.)

- table="expression",
- cancer_type=["Renal cell carcinoma", "Prostate", "Adenoid cystic carcinoma", "Breast", "Thyroid", "Testicular", "Hepatocellular carcinoma", "Melanoma"],
- genes=list(genes_df['gene']),
- source = "PDX"
(For example, if you use the data_request.ipynb in Python, these are the options in the df, location = request_data() function.)

## 3. Read data

In [6]:
# change to where you saved your .csv file and file name
df  <- fread("../data/pdx_expression_gene_list_multi_cancer.csv")
df  <- as.data.frame(df)
df$gene  <- as.character(df$gene)  # each column is a list - specify data type
df$model  <- as.character(df$model)
df$tumor_type  <- as.character(df$tumor_type)
df$log.tpm  <- as.numeric(df$log.tpm)
head(df, 2)
print(dim(df))

,gene,model,model_name,log.tpm,z,fold,tumor_type
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
1,ACSM3,CTG-3501,CTG-3501,2.7411490,-0.1693172,0.8765697,Breast
2,COPZ2,CTG-3501,CTG-3501,0.7995996,-0.9946150,0.1851744,Breast


[1] 82800     7


In [7]:
# Pivot the table
log_tpm <- df %>% 
    select(c('gene', 'model', 'log.tpm'))  %>% 
    pivot_wider(names_from = gene, values_from = log.tpm)  %>% 
    column_to_rownames('model')
log_tpm  <- log_tpm[complete.cases(log_tpm),]  # remove rows with missing values
log_tpm[1:2, 1:10]
print(dim(log_tpm))

ERROR: Error in (function (classes, fdef, mtable) : unable to find an inherited method for function 'select' for signature '"data.frame"'


## 4. Select the power

In [ ]:
# Choose a set of soft-thresholding powers
powers = c(c(1:10), seq(from = 12, to = 20, by = 2))

# Call the network topology analysis function
sft = pickSoftThreshold(
  log_tpm,             # <= Input data
  powerVector = powers,
  verbose = 5
  )

In [ ]:
par(mfrow = c(1,2));
cex1 = 0.9;

plot(sft$fitIndices[, 1],
     -sign(sft$fitIndices[, 3]) * sft$fitIndices[, 2],
     xlab = "Threshold",
     ylab = "R2",
     main = paste("Scale independence")
)
text(sft$fitIndices[, 1],
     -sign(sft$fitIndices[, 3]) * sft$fitIndices[, 2],
     labels = powers, cex = cex1, col = "red"
)
abline(h = 0.90, col = "red")
plot(sft$fitIndices[, 1],
     sft$fitIndices[, 5],
     xlab = "Threshold",
     ylab = "Mean Connectivity",
     type = "n",
     main = paste("Mean connectivity")
)
text(sft$fitIndices[, 1],
     sft$fitIndices[, 5],
     labels = powers,
     cex = cex1, col = "red")

According to the above two plots, we should pick power=14.

## 5. Create the network

In [ ]:
picked_power = 9
temp_cor <- cor       
cor <- WGCNA::cor         # Force it to use WGCNA cor function (fix a namespace conflict issue)
netwk <- blockwiseModules(log_tpm,                # <= input here

                          # == Adjacency Function ==
                          power = picked_power,                # <= power here
                          networkType = "signed",

                          # == Tree and Block Options ==
                          deepSplit = 2,
                          pamRespectsDendro = F,
                          minModuleSize = 30,
                          maxBlockSize = 4000,

                          # == Module Adjustments ==
                          reassignThreshold = 0,
                          mergeCutHeight = 0.25,

                          # == Archive the run results in TOM file (saves time) == 
                          saveTOMs = T,
                          saveTOMFileBase = "ER",

                          # == Output Options
                          numericLabels = T,
                          verbose = 3)
cor <- temp_cor     # Return cor function to original namespace

In [ ]:
# Convert labels to colors for plotting
mergedColors = labels2colors(netwk$colors)
# Plot the dendrogram and the module colors underneath
options(repr.plot.width=10, repr.plot.height=4, repr.plot.res=300)
plotDendroAndColors(
  netwk$dendrograms[[1]],
  mergedColors[netwk$blockGenes[[1]]],
  "Module colors",
  dendroLabels = FALSE,
  hang = 0.03,
  addGuide = TRUE,
  guideHang = 0.05 )

## 6. Relate the modules

In [ ]:
module_df <- data.frame(
  gene_id = names(netwk$colors),
  colors = labels2colors(netwk$colors)
)

In [ ]:
head(module_df)

In [ ]:
# Get Module Eigengenes per cluster
MEs0 <- moduleEigengenes(log_tpm, mergedColors)$eigengenes

# Reorder modules so similar modules are next to each other
MEs0 <- orderMEs(MEs0)
module_order <- names(MEs0) %>% gsub("ME","", .)

# Add model names
MEs0$model = row.names(MEs0)

head(MEs0, 2)

In [ ]:
# tidy & plot data
mME = MEs0[1: 30,] %>%  # choose the first numbers of models here - you may select models of interest
  pivot_longer(-model) %>%
  mutate(
    name = gsub("ME", "", name),
    name = factor(name, levels = module_order)
  )

mME %>% ggplot(., aes(x=model, y=name, fill=value)) +
  geom_tile() +
  theme_bw() +
  scale_fill_gradient2(
    low = "blue",
    high = "red",
    mid = "white",
    midpoint = 0,
    limit = c(-1,1)) +
  theme(axis.text.x = element_text(angle=90)) +
  labs(title = "Module-model Relationships", y = "Modules", fill="corr")